In [2]:
import pandas as pd
import json
from pathlib import Path
import sys
sys.path.append("../")
from utils import read_meta, MetaData, KeypointInfo

In [3]:
def json_to_df(path:Path)->pd.DataFrame:
    cols = ["image_path", "frame_no", "model_name"]
    # cols.extend([o for o in KeypointInfo._fields])
    cols.extend([f"kp_{i}" for i in range(12)])
    df =  pd.DataFrame(columns=cols)

    for i, fn in enumerate(path.glob("*.json")):
        meta_data = read_meta(fn)
        image_path = meta_data.source["image"]
        model_name = meta_data.source["object_id"]
        frame_no = int(image_path.split("/")[-1].split("_")[-1].split(".")[0])
        d = [image_path, frame_no, model_name]
        d.extend(meta_data.keypoints)
        df.loc[i] = d
        # print(meta_data)
        # break
    return df


In [40]:
input_data_name="IMG_4551_CHRIS"
df_ref = json_to_df(Path(f"../results/{input_data_name}/annotated/ref/"))
df_test = json_to_df(Path(f"../results/{input_data_name}/annotated/test/"))

df_ref = df_ref.add_prefix("ref_")
df_test = df_test.add_prefix("test_")
df_comb = pd.concat([df_ref,df_test], axis=1)
assert all(df_comb[["ref_frame_no", "test_frame_no"]].apply(lambda x: x[0]==x[1], axis=1))

/Users/benjamin/.pyenv/versions/3.8.6/envs/ml38/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



In [20]:
from functools import partial
def pixel_distance(x_ratio, y_ratio, p1, p2)->float:
    return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5

w,h = Image.open(f"../{df_comb['ref_image_path'].loc[0]}").size
pixel_distance = partial(pixel_distance, x_ratio = (160/w), y_ratio=(160/h))
for i in range(12):
    df_comb[f"diff_kp_{i}"] = df_comb.apply(lambda x: pixel_distance(p1=x[f"ref_kp_{i}"],p2=x[f"test_kp_{i}"]), axis=1)

In [38]:
pd.options.plotting.backend = "plotly"
cols=[]
for i in range(12):
    cols.append(f"diff_kp_{i}")
df_comb[cols].plot()
fig = df_comb[cols].describe().loc[["mean", "min", "max", "75%"]].plot(kind="bar", barmode='group', title=f"{input_data_name} - {ref_model_name} {test_model_name}")


In [39]:
# fig.add_annotation()
fig.write_html("tmp.html")

In [22]:
df_comb[cols].plot()

In [15]:
worst_frames={}
from PIL import Image, ImageDraw
for c in cols:
    worst_frames[c]=df_comb[c].nlargest(4).index.tolist()
for k,v in worst_frames.items():
    print(f"{k}: {v}")
    worsts_image = Image.new("RGB", (len(v)*160, 160*2))

    x_offset = 0
    for i in v:
        print(df_comb['ref_image_path'].loc[i])
        test_image_path = df_comb['ref_image_path'].loc[i].split("/")
        test_image_path.insert(-1, 'annotated/test')
        annotated_image_path = "/".join(test_image_path)
        wi = Image.open(f"../{annotated_image_path}")
        w, h = wi.size
        wi = wi.resize((160,160))
        draw = ImageDraw.Draw(wi)
        draw.text((20,20),"Bouldrama", (255,255,255))
        x, y = df_comb[k.replace("diff", "test")].loc[i]
        x, y = (x/w)*160, (y/h)*160
        # draw.rectangle((10, 10, 100, 100), fill ="#ffff33", outline ="red")
        pointer_size=2
        draw.rectangle((x-pointer_size, y-pointer_size, x+pointer_size, y+pointer_size), fill ="#ffff33", outline ="red")

        worsts_image.paste(
            wi,
            (x_offset, 0),
        )
        x_offset += 160
    x_offset = 0
    for i in v:
        test_image_path = df_comb['ref_image_path'].loc[i].split("/")
        test_image_path.insert(-1, 'annotated/ref')
        annotated_image_path = "/".join(test_image_path)
        wi = Image.open(f"../{annotated_image_path}")
        w, h = wi.size
        wi = wi.resize((160,160))
        draw = ImageDraw.Draw(wi)
        draw.text((20,20),"Ref", (255,255,255))
        x, y = df_comb[k.replace("diff", "ref")].loc[i]
        x, y = (x/w)*160, (y/h)*160
        # draw.rectangle((10, 10, 100, 100), fill ="#ffff33", outline ="red")
        pointer_size=2
        draw.rectangle((x-pointer_size, y-pointer_size, x+pointer_size, y+pointer_size), fill ="#ffff33", outline ="red")
        worsts_image.paste(
            wi,
            (x_offset, 160),
        )
        x_offset += 160
    worsts_image.save(f"{k}.png")

diff_kp_0: [523, 39, 529, 328]
results/IMG_4551_CHRIS/frame_425.png
results/IMG_4551_CHRIS/frame_057.png
results/IMG_4551_CHRIS/frame_536.png
results/IMG_4551_CHRIS/frame_423.png
diff_kp_1: [523, 149, 332, 328]
results/IMG_4551_CHRIS/frame_425.png
results/IMG_4551_CHRIS/frame_476.png
results/IMG_4551_CHRIS/frame_474.png
results/IMG_4551_CHRIS/frame_423.png
diff_kp_2: [523, 406, 602, 308]
results/IMG_4551_CHRIS/frame_425.png
results/IMG_4551_CHRIS/frame_443.png
results/IMG_4551_CHRIS/frame_404.png
results/IMG_4551_CHRIS/frame_272.png
diff_kp_3: [523, 98, 441, 32]
results/IMG_4551_CHRIS/frame_425.png
results/IMG_4551_CHRIS/frame_426.png
results/IMG_4551_CHRIS/frame_418.png
results/IMG_4551_CHRIS/frame_283.png
diff_kp_4: [149, 523, 422, 211]
results/IMG_4551_CHRIS/frame_476.png
results/IMG_4551_CHRIS/frame_425.png
results/IMG_4551_CHRIS/frame_475.png
results/IMG_4551_CHRIS/frame_505.png
diff_kp_5: [523, 14, 571, 258]
results/IMG_4551_CHRIS/frame_425.png
results/IMG_4551_CHRIS/frame_578.pn

In [35]:
x,y = df_comb["test_kp_0"].loc[0]